In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [2]:
path = "C:/Users/user/Documents/GitHub/Visual-Analytics/src/static/dataset/"
ctg = pd.read_csv(path+"prod_cat_info.csv") #categories csv
cst = pd.read_csv(path+"Customer.csv")
tr = pd.read_csv(path+"Transactions.csv")
data = pd.read_csv(path+"full_data.csv")

In [47]:
ctg.head()

,prod_cat_code,prod_cat,prod_sub_cat_code,prod_subcat
0,1,Clothing,4,Mens
1,1,Clothing,1,Women
2,1,Clothing,3,Kids
3,2,Footwear,1,Mens
4,2,Footwear,3,Women


In [48]:
cst.head()

,customer_Id,DOB,Gender,city_code
0,268408,02-01-1970,M,4.0
1,269696,07-01-1970,F,8.0
2,268159,08-01-1970,F,8.0
3,270181,10-01-1970,F,2.0
4,268073,11-01-1970,M,1.0


In [49]:
tr.head()

,transaction_id,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type
0,80712190438,270351,28-02-2014,1,1,-5,-772,405.300,-4265.300,e-Shop
1,29258453508,270384,27-02-2014,5,3,-5,-1497,785.925,-8270.925,e-Shop
2,51750724947,273420,24-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop
3,93274880719,271509,24-02-2014,11,6,-3,-1363,429.345,-4518.345,e-Shop
4,51750724947,273420,23-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop


In [50]:
tr['Qty'] = np.abs(tr['Qty'])
tr['total_amt'] = np.abs(tr['total_amt'])
tr.head()

,transaction_id,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type
0,80712190438,270351,28-02-2014,1,1,5,-772,405.300,4265.300,e-Shop
1,29258453508,270384,27-02-2014,5,3,5,-1497,785.925,8270.925,e-Shop
2,51750724947,273420,24-02-2014,6,5,2,-791,166.110,1748.110,TeleShop
3,93274880719,271509,24-02-2014,11,6,3,-1363,429.345,4518.345,e-Shop
4,51750724947,273420,23-02-2014,6,5,2,-791,166.110,1748.110,TeleShop


In [51]:
#Merge transaction data with category information
tr_merged = pd.merge(tr, ctg,  how='left', left_on=['prod_cat_code','prod_subcat_code'], 
right_on = ['prod_cat_code','prod_sub_cat_code'])

#Merge with customer information
tr_final = pd.merge(tr_merged, cst,  how='left', left_on=['cust_id'], 
right_on = ['customer_Id'])
tr_final = tr_final.sort_values(by=["tran_date"])

#tr_final.to_csv(path+'full_data.csv', index=False)
tr_final.head()

,transaction_id,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type,prod_cat,prod_sub_cat_code,prod_subcat,customer_Id,DOB,Gender,city_code
16050,47933752822,270192,1/1/2012,3,1,4,215,90.300,950.300,e-Shop,Clothing,3,Kids,270192,01-06-1987,F,4.0
16052,35798683065,271913,1/1/2012,1,1,1,229,24.045,253.045,MBR,Clothing,1,Women,271913,15-10-1985,F,2.0
16053,30078154552,269151,1/1/2012,3,2,1,456,47.880,503.880,e-Shop,Footwear,3,Women,269151,29-04-1981,F,2.0
16054,57039573748,268282,1/1/2012,4,1,1,1024,107.520,1131.520,MBR,Clothing,4,Mens,268282,22-12-1970,F,1.0
16055,41901986246,269269,1/1/2012,7,5,2,1122,235.620,2479.620,MBR,Books,7,Fiction,269269,04-02-1981,M,1.0


In [20]:
grouped_tr = tr.groupby(['prod_subcat_code', 'prod_cat_code'])["Qty", "total_amt"].apply(lambda x : x.astype(int).sum())
grouped_tr.head()

Qty  total_amt
prod_subcat_code prod_cat_code                 
1                1              3085    2677312
                 2              2814    2439523
                 4              2976    2623618
2                6              3073    2651658
3                1              2961    2644253

In [28]:
merged_tr = pd.merge(grouped_tr, ctg,  how='outer', left_on=['prod_cat_code','prod_subcat_code'], 
right_on = ['prod_cat_code','prod_sub_cat_code'])
merged_tr = merged_tr.sort_values(by=["prod_cat_code","prod_sub_cat_code"])
merged_tr.head()

,prod_cat_code,Qty,total_amt,prod_cat,prod_sub_cat_code,prod_subcat
0,1,3085,2677312,Clothing,1,Women
4,1,2961,2644253,Clothing,3,Kids
7,1,2916,2574840,Clothing,4,Mens
1,2,2814,2439523,Footwear,1,Mens
5,2,3075,2576409,Footwear,3,Women


Use this in js:

date = //read Python generated data here

var nestedData = d3.nest()
 .key(function(d) { return d.category; })
 .key(function(d) { return d.subcategory; })
 .entries(data);

In [3]:
data.head()

,transaction_id,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type,prod_cat,prod_sub_cat_code,prod_subcat,customer_Id,DOB,Gender,city_code
0,36332303449,268624,2011-01-02,10,6,4,-295,123.900,1303.900,Flagship store,Home and kitchen,10,Kitchen,268624,18-09-1986,F,1.0
1,25374972356,268904,2011-01-02,2,6,5,821,431.025,4536.025,MBR,Home and kitchen,2,Furnishing,268904,25-05-1990,M,5.0
2,15662366857,272756,2011-01-02,5,3,3,527,166.005,1747.005,e-Shop,Electronics,5,Computers,272756,03-12-1975,F,9.0
3,28972634039,275227,2011-01-02,9,3,1,-334,35.070,369.070,MBR,Electronics,9,Cameras,275227,23-09-1989,M,4.0
4,60041644943,267309,2011-01-02,3,2,1,392,41.160,433.160,Flagship store,Footwear,3,Women,267309,08-04-1973,F,1.0


In [43]:
gr_data = data.groupby(['cust_id', 'prod_subcat'])["Qty"].apply(lambda x : x.astype(int).sum()).to_frame()
gr_data = gr_data.reset_index()
gr_data.head()


,cust_id,prod_subcat,Qty
0,266783,Mens,12
1,266783,Non-Fiction,2
2,266784,Fiction,5
3,266784,Mobiles,2
4,266784,Non-Fiction,3


In [85]:
gr_data.columns
print(gr_data.cust_id.tolist())

[266783, 266784, 266785, 266788, 266794, 266799, 266803, 266804, 266805, 266806, 266807, 266809, 266810, 266812, 266813, 266814, 266815, 266816, 266817, 266818, 266819, 266820, 266821, 266822, 266823, 266824, 266825, 266827, 266829, 266830, 266831, 266833, 266835, 266836, 266838, 266841, 266842, 266843, 266844, 266845, 266846, 266847, 266848, 266849, 266851, 266852, 266853, 266854, 266855, 266856, 266857, 266858, 266860, 266861, 266862, 266863, 266864, 266865, 266866, 266867, 266870, 266872, 266875, 266878, 266881, 266882, 266883, 266885, 266886, 266887, 266888, 266889, 266891, 266892, 266894, 266895, 266896, 266897, 266900, 266901, 266902, 266903, 266904, 266906, 266907, 266910, 266911, 266915, 266916, 266917, 266918, 266919, 266920, 266921, 266922, 266923, 266927, 266929, 266930, 266931, 266934, 266935, 266936, 266937, 266939, 266940, 266941, 266942, 266943, 266944, 266945, 266950, 266952, 266953, 266956, 266957, 266958, 266959, 266960, 266964, 266966, 266972, 266974, 266975, 266976,

In [46]:
gr_data = gr_data.pivot(index="cust_id", columns="prod_subcat", values="Qty")
gr_data = gr_data.fillna(0)
gr_data.head()

prod_subcat,Academic,Audio and video,Bath,Cameras,Children,Comics,Computers,DIY,Fiction,Furnishing,Kids,Kitchen,Mens,Mobiles,Non-Fiction,Personal Appliances,Tools,Women
cust_id,,,,,,,,,,,,,,,,,,
266783,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,2.0,0.0,0.0,0.0
266784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0
266785,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,7.0,10.0,11.0,0.0,0.0,0.0,0.0,0.0
266788,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
266794,4.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,2.0,0.0,0.0,4.0,0.0,8.0


In [57]:
# Get feature names
features = gr_data.columns[1:].tolist()
print(features)

# Get customer ids
customers = gr_data["cust_id"].tolist()
print(customers)

['Academic', 'Audio and video', 'Bath', 'Cameras', 'Children', 'Comics', 'Computers', 'DIY', 'Fiction', 'Furnishing', 'Kids', 'Kitchen', 'Mens', 'Mobiles', 'Non-Fiction', 'Personal Appliances', 'Tools', 'Women']
[266783, 266784, 266785, 266788, 266794, 266799, 266803, 266804, 266805, 266806, 266807, 266809, 266810, 266812, 266813, 266814, 266815, 266816, 266817, 266818, 266819, 266820, 266821, 266822, 266823, 266824, 266825, 266827, 266829, 266830, 266831, 266833, 266835, 266836, 266838, 266841, 266842, 266843, 266844, 266845, 266846, 266847, 266848, 266849, 266851, 266852, 266853, 266854, 266855, 266856, 266857, 266858, 266860, 266861, 266862, 266863, 266864, 266865, 266866, 266867, 266870, 266872, 266875, 266878, 266881, 266882, 266883, 266885, 266886, 266887, 266888, 266889, 266891, 266892, 266894, 266895, 266896, 266897, 266900, 266901, 266902, 266903, 266904, 266906, 266907, 266910, 266911, 266915, 266916, 266917, 266918, 266919, 266920, 266921, 266922, 266923, 266927, 266929, 266

In [60]:
# Separating out the features
x = gr_data.loc[:, features].values
print(x)

# Standardizing the features
x = StandardScaler().fit_transform(x)
print(x)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 4. 0.]
 [3. 0. 0. ... 0. 0. 0.]]
[[-0.34230802 -0.35114004 -0.35682812 ... -0.34670981 -0.35973491
  -0.61358852]
 [-0.34230802 -0.35114004 -0.35682812 ... -0.34670981 -0.35973491
  -0.61358852]
 [-0.34230802 -0.35114004 -0.35682812 ... -0.34670981 -0.35973491
  -0.61358852]
 ...
 [-0.34230802 -0.35114004 -0.35682812 ... -0.34670981 -0.35973491
  -0.61358852]
 [-0.34230802 -0.35114004 -0.35682812 ... -0.34670981  2.13092891
  -0.61358852]
 [ 1.59673308 -0.35114004 -0.35682812 ... -0.34670981 -0.35973491
  -0.61358852]]


In [64]:
#Applying PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x) #vector of principal components for each cust_id
principalDf = pd.DataFrame(data = principalComponents, columns = ['pc 1', 'pc 2'])
principalDf = pd.concat([gr_data['cust_id'], principalDf], axis = 1)
principalDf.head()

,cust_id,pc 1,pc 2
0,266783,0.187507,-1.420041
1,266784,-0.925002,-0.443045
2,266785,0.293841,-3.009612
3,266788,0.083269,-1.119107
4,266794,-0.162038,1.255707


In [66]:
#Running K-Means
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(x)
print(kmeans.labels_)
print(kmeans.cluster_centers_)

[1 2 1 ... 1 2 3]
[[-4.53137621e-03 -4.21806386e-02  2.71694663e+00 -1.00315093e-02
  -3.27062805e-02 -7.33704793e-02 -3.09418020e-02  5.37733242e-02
   5.97563123e-02 -1.55880256e-01 -1.32913687e-02 -7.03851715e-03
   3.53657243e-02  8.09446048e-02 -1.51437898e-01 -4.59114839e-02
   6.03546646e-03 -4.43202496e-02]
 [ 3.74218791e-03  7.73110911e-03 -2.90443780e-01 -6.93157792e-04
  -1.00597505e-03 -5.12122708e-03  1.90016879e-03  4.92752762e-03
  -6.80476567e-03 -3.02170293e-01 -2.92653113e-03  8.98465156e-03
  -5.01310499e-03  3.00264087e-03 -2.93835596e-01  3.83404354e-03
   6.07358063e-03  1.38406174e-02]
 [ 2.74131242e-02  1.73071603e-02 -1.99307674e-01  3.58609044e-02
   1.76091300e-02  6.10601231e-02  3.25628503e-02 -6.16527895e-02
   8.18179672e-03 -1.36061226e-01 -5.12983541e-04 -4.22033155e-02
  -5.53154266e-02 -4.79650603e-02  2.77863865e+00 -6.09198011e-02
  -3.55898737e-02  1.51960137e-02]
 [-5.21642600e-02 -3.59890675e-02 -2.42087895e-01 -1.91210666e-02
   2.42994619e-02  

In [83]:
#Append labels on dataframe and write it to csv file
labels = pd.Series(kmeans.labels_)
#print(labels)
clusteredDf = pd.concat([principalDf, labels], axis = 1)
clusteredDf = clusteredDf.rename(columns={"cust_id": "cust_id", "pc 1": "pc_1", "pc 2": "pc_2", 0: "cluster"})
clusteredDf.head()
clusteredDf.to_csv(path+'pca_kmeans_data.csv', index=False)
